
### 가장 대여량이 높은 대여소 TOP 1
SELECT rental_station_id, AVG(rental_count)
FROM rental_daily_station
GROUP BY rental_station_id
HAVING AVG(rental_count)
ORDER By AVG(rental_count) desc

### 가장 대여량이 높은 대여소 "ST-2031" 전부
SELECT *
FROM rental_hourly_station
WHERE rental_station_id = "ST-2031"

### 가장 대여량이 높은 대여소 10곳 전부
SELECT *
FROM rental_hourly_station r, 
    (SELECT rental_station_id stations, AVG(rental_count)
    FROM rental_daily_station
    GROUP BY rental_station_id
    HAVING AVG(rental_count)
    ORDER By AVG(rental_count) desc
    LIMIT 10) d
WHERE r.rental_station_id = d.stations

### 대여소 + 날씨 "ST-2031" 전부
SELECT *
FROM weather_hourly w, (SELECT *
                        FROM rental_hourly_station
                        WHERE rental_station_id = "ST-2031") r
WHERE (r.rental_date = w.weather_date) AND (r.rental_hour = w.weather_hour)

### 통합 검색 - "ST-2031"
SELECT w.weather_date, w.weather_hour, r.rental_station_id, r.rental_count, 
        c.weekday_int, c.weekday_name, c.is_holiday
        temp, rain, humid, wind, sunlight_direct, radiation_total, snow
FROM weather_hourly w, calendar c, (SELECT *
                                    FROM rental_hourly_station
                                    WHERE rental_station_id = "ST-2031") r
WHERE (r.rental_date = w.weather_date) 
    AND (r.rental_date = c.rental_date) 
    AND (r.rental_hour = w.weather_hour)

# DB 에서 정보 불러오기

In [8]:
import mysql.connector


remote = mysql.connector.connect (
    host = "jinhyuk2db.c94oygiiukcg.ap-northeast-2.rds.amazonaws.com",
    port = 3306,
    user = "root",
    password = "jinhyuk2dacibul",
    database = "project1"
)

limit = 1

cursor = remote.cursor(buffered=True)
cursor.execute(f"""SELECT w.weather_date, w.weather_hour, r.rental_station_id, r.rental_count, 
                            c.weekday_int, c.weekday_name, c.is_holiday,
                            temp, rain, humid, wind, sunlight_direct, radiation_total, snow
                    FROM weather_hourly w, calendar c, 
                        (SELECT *
                        FROM rental_hourly_station r, 
                            (SELECT rental_station_id stations, AVG(rental_count)
                            FROM rental_daily_station
                            GROUP BY rental_station_id
                            HAVING AVG(rental_count)
                            ORDER By AVG(rental_count) desc
                            LIMIT {limit}) d
                        WHERE r.rental_station_id = d.stations) r
                    WHERE (r.rental_date = w.weather_date) 
                        AND (r.rental_date = c.rental_date) 
                        AND (r.rental_hour = w.weather_hour)""")

result = cursor.fetchall()

remote.close()

In [7]:
result[0]

(datetime.date(2024, 1, 1),
 0,
 'ST-2031',
 7,
 0,
 'Monday',
 1,
 0.0,
 97.0,
 1.9,
 0.0,
 0.0,
 2.4)

# df 테이블 변환

In [9]:
import pandas as pd

df = pd.DataFrame(result, columns = ['일자', '시간(시)', '대여소ID', '대여량', '요일ID', '요일', '휴일', 
                                     '기온(℃)', '강수량(mm)', '습도(%rh)', '풍속(m/s)', '일조(hr)', '일사(MJ/m2)', '적설(cm)'])
df

,일자,시간(시),대여소ID,대여량,요일ID,요일,휴일,기온(℃),강수량(mm),습도(%rh),풍속(m/s),일조(hr),일사(MJ/m2),적설(cm)
0,2024-01-01,0,ST-2031,7,0,Monday,1,0.6,0.0,97.0,1.9,0.0,0.00,2.4
1,2024-01-01,1,ST-2031,4,0,Monday,1,0.5,0.0,97.0,2.2,0.0,0.00,2.4
2,2024-01-01,5,ST-2031,5,0,Monday,1,0.3,0.0,99.0,1.2,0.0,0.00,2.4
3,2024-01-01,6,ST-2031,1,0,Monday,1,0.5,0.0,99.0,2.0,0.0,0.00,2.4
4,2024-01-01,7,ST-2031,3,0,Monday,1,0.4,0.0,98.0,2.1,0.0,0.00,2.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8041,2024-12-31,3,ST-2031,1,1,Tuesday,0,2.0,0.0,53.0,3.2,0.0,0.00,0.0
8042,2024-12-31,5,ST-2031,2,1,Tuesday,0,0.2,0.0,49.0,2.7,0.0,0.00,0.0
8043,2024-12-31,17,ST-2031,40,1,Tuesday,0,2.5,0.0,37.0,3.4,1.0,0.42,0.0
8044,2024-12-31,21,ST-2031,15,1,Tuesday,0,-0.7,0.0,51.0,1.0,0.0,0.00,0.0


In [12]:
df['일조(hr)'].unique()

array([0. , 0.5, 1. , 0.8, 0.1, 0.2, 0.4, 0.9, 0.7, 0.3, 0.6])